CTDC005 ctDNA Analysis 

How this patient was selected?

Using the command - 
(echo -e "PatientID\tSample Name\tIsolate\tTreatment Stage\tExperiment"; csvcut -c "Sample Name","isolate","Experiment" ~/Downloads/SraRunTable.csv | sed '1d' | awk -F',' '{
  if ($1 ~ /^PBMC_/) stage="Pre-treatment";
  else if ($1 ~ /^FFPE_/) stage="Post-treatment";
  else if ($1 ~ /-[0-9]+$/) stage="During treatment";
  else stage="Baseline";
  split($1, a, /[^0-9]/);
  for (i in a) if (a[i] ~ /^[0-9]+$/) {id=a[i]; break}
  print id "\t" $1 "\t" $2 "\t" stage "\t" $3
}' | sort -k1,1n | grep '^005')
PatientID       Sample Name     Isolate Treatment Stage Experiment
005     C_fw005 blood   Baseline        SRX10351538
005     C_fw005-2       blood   During treatment        SRX10351537
005     FFPE_C_fw005    tissue biopsy   Post-treatment  SRX10351703
005     PBMC_C_fw005    blood   Pre-treatment   SRX10351410

Metadata file: SraRunTable.csv was downloaded from https://www.ncbi.nlm.nih.gov/Traces/study/?acc=PRJNA714799&o=acc_s%3Aa by clicking 'Metadata' under 'Download'

CTDC05 in Table 4 in the paper - https://www.nature.com/articles/s41598-021-95345-4/tables/4
has Gene KRAS missense variant with Pathogenic/Likely_pathogenic 



CTDC005 ctDNA Analysis 

How this patient was selected?

Using the command - 
(echo -e "PatientID\tSample Name\tIsolate\tTreatment Stage\tExperiment"; csvcut -c "Sample Name","isolate","Experiment" ~/Downloads/SraRunTable.csv | sed '1d' | awk -F',' '{
  if ($1 ~ /^PBMC_/) stage="Pre-treatment";
  else if ($1 ~ /^FFPE_/) stage="Post-treatment";
  else if ($1 ~ /-[0-9]+$/) stage="During treatment";
  else stage="Baseline";
  split($1, a, /[^0-9]/);
  for (i in a) if (a[i] ~ /^[0-9]+$/) {id=a[i]; break}
  print id "\t" $1 "\t" $2 "\t" stage "\t" $3
}' | sort -k1,1n | grep '^005')

Output 

PatientID       Sample Name     Isolate Treatment Stage Experiment

005     C_fw005 blood   Baseline        SRX10351538

005     C_fw005-2       blood   During treatment        SRX10351537

005     FFPE_C_fw005    tissue biopsy   Post-treatment  SRX10351703

005     PBMC_C_fw005    blood   Pre-treatment   SRX10351410

Metadata file: SraRunTable.csv was downloaded from https://www.ncbi.nlm.nih.gov/Traces/study/?acc=PRJNA714799&o=acc_s%3Aa by clicking 'Metadata' under 'Download'

CTDC05 in Table 4 in the paper - https://www.nature.com/articles/s41598-021-95345-4/tables/4
has Gene KRAS missense variant with Pathogenic/Likely_pathogenic 



In [ ]:
# commands of fastqc and multiqc are run in the terminal, not in this notebook
    # fastqc /Volumes/T7/GTMBioinfo/raw/*.fastq -o docs/
    # multiqc docs/ -o docs/
    # Save the multiqc report as PDF in docs/ folder from html report

# multi qc report - https://github.com/justin-mbca/GTMBioinfo/blob/main/docs/MultiQCReport.pdf
# From the report, the quality scores are good in overall and there is low adapter content across samples.
    # Therefore, no trimming is needed for these samples.
# The duplicate rates are high across samples, which might be due to deep sequencing with low input DNA amounts.
    # Therefore, the duplicates will be removed during alignment and variant calling steps.    
    # This should be considered when interpreting variant allele frequencies (VAFs) later in the analysis.

Alignment 
# Create GRCh38 reference genome
   1. Download chromosome 12 of reference fasta file - wget -P /Volumes/T7/GTMBioinfo/reference/ ftp://ftp.ensembl.org/pub/release-115/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.12.fa.gz
   2. Extract zip file -  gunzip /Volumes/T7/GTMBioinfo/reference/Homo_sapiens.GRCh38.dna.chromosome.12.fa.gz
   3. Create BWA index file - bwa index /Volumes/T7/GTMBioinfo/reference/Homo_sapiens.GRCh38.dna.chromosome.12.fa
   4. Create FASTA index - samtools faidx /Volumes/T7/GTMBioinfo/reference/Homo_sapiens.GRCh38.dna.chromosome.12.fa
# Alignment
   1. create bash script in alignment.sh file 
   2. Loop through each sample's fastq files to perform alignment and bam conversion/indexing
      1. bwa mem is used for mapping medium length PE reads. 
      2. bam binamry file, sorting and indexing for high performance required by downstream steps. 

# Variant calling

for sample in C_fw005 C_fw005-2 FFPE_C_fw005 PBMC_C_fw005; do
  gatk MarkDuplicates \
    -I /Volumes/T7/GTMBioinfo/aligned/${sample}.chr12.sorted.bam \
    -O /Volumes/T7/GTMBioinfo/aligned/${sample}.chr12.dedup.bam \
    -M /Volumes/T7/GTMBioinfo/aligned/${sample}.metrics.txt
  samtools index /Volumes/T7/GTMBioinfo/aligned/${sample}.chr12.dedup.bam
done


In [ ]:
# Verify samples and collect FASTQ links from local metadata file
import pandas as pd

meta = pd.read_csv('PRJNA714799_plus.tsv', sep='	')
aliases = ['CTC030', 'CTC030-2', 'CTC030-6', 'PBMC_CTC030', 'FFPE_CTC030']
rows = meta[meta['sample_alias'].isin(aliases)].copy()
rows = rows[['run_accession','sample_accession','sample_alias','fastq_ftp','library_layout']]
print(rows.to_string(index=False))

print('
Copyable download commands:')
for _, r in rows.iterrows():
    run = r['run_accession']
    alias = r['sample_alias']
    links = str(r['fastq_ftp']).split(';') if pd.notna(r['fastq_ftp']) else []
    links = [l.strip() for l in links if l.strip()]
    print(f
)
    for l in links:
        url = l if l.startswith('ftp://') else f'ftp://{l}'
        print(f'wget {url}')

Download FASTQ files (command-line)

Blood timepoints (primary ctDNA analysis):

```bash
# CTC030 (SRR13973811)
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR139/011/SRR13973811/SRR13973811_1.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR139/011/SRR13973811/SRR13973811_2.fastq.gz

# CTC030-2 (SRR13973813)
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR139/013/SRR13973813/SRR13973813_1.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR139/013/SRR13973813/SRR13973813_2.fastq.gz

# CTC030-6 (SRR13973812)
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR139/012/SRR13973812/SRR13973812_1.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR139/012/SRR13973812/SRR13973812_2.fastq.gz
```

Optional background controls:

```bash
# PBMC (SRR13974115)
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR139/015/SRR13974115/SRR13974115_1.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR139/015/SRR13974115/SRR13974115_2.fastq.gz

# FFPE tissue (SRR13974091)
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR139/091/SRR13974091/SRR13974091_1.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR139/091/SRR13974091/SRR13974091_2.fastq.gz
```

Storage note: I keep large files on /Volumes/T7 (external SSD) to avoid filling the system disk.

Reference genome setup (GRCh38)

For this example, I’ll focus on KRAS hotspots on chr12. This keeps the analysis lightweight and fast. You can extend to other genes later.

```bash
# Download GRCh38 chromosome 12 (Ensembl)
mkdir -p /Volumes/T7/GTMBioinfo/reference
cd /Volumes/T7/GTMBioinfo/reference
curl -O https://ftp.ensembl.org/pub/release-112/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.12.fa.gz
gunzip Homo_sapiens.GRCh38.dna.chromosome.12.fa.gz

# Index for BWA and samtools
bwa index Homo_sapiens.GRCh38.dna.chromosome.12.fa
samtools faidx Homo_sapiens.GRCh38.dna.chromosome.12.fa
```

Align reads (example for one sample)

```bash
# Example: CTC030 baseline
bwa mem -t 4 \
  -R '@RG	ID:SRR13973811	SM:CTC030	PL:ILLUMINA' \
  /Volumes/T7/GTMBioinfo/reference/Homo_sapiens.GRCh38.dna.chromosome.12.fa \
  SRR13973811_1.fastq.gz \
  SRR13973811_2.fastq.gz | \
samtools sort -@ 4 -o /Volumes/T7/GTMBioinfo/alignments/CTC030.sorted.bam -

samtools index /Volumes/T7/GTMBioinfo/alignments/CTC030.sorted.bam
```

I repeat the same pattern for CTC030-2 and CTC030-6, changing the IDs and filenames accordingly.

Quality control (flagstat)

```bash
samtools flagstat /Volumes/T7/GTMBioinfo/alignments/CTC030.sorted.bam
samtools flagstat /Volumes/T7/GTMBioinfo/alignments/CTC030-2.sorted.bam
samtools flagstat /Volumes/T7/GTMBioinfo/alignments/CTC030-6.sorted.bam
```

I paste the raw outputs here, then write a simple plain-text summary table by hand.

Variant calling (bcftools) — KRAS hotspot window

KRAS on GRCh38 is on chr12. I use a small window around codons 12/13 and 61.

```bash
# Codon 12/13 window ~ chr12:25,398,000-25,399,000 (GRCh38 G12/G13 region)
# Codon 61 window   ~ chr12:25,380,000-25,390,000

# Example call for one BAM against codon 12/13 window
bcftools mpileup -Ou \
  -f /Volumes/T7/GTMBioinfo/reference/Homo_sapiens.GRCh38.dna.chromosome.12.fa \
  --regions 12:25398000-25399000 \
  -Q 20 -q 20 \
  /Volumes/T7/GTMBioinfo/alignments/CTC030.sorted.bam | \
bcftools call -mv -Oz -o /Volumes/T7/GTMBioinfo/vcf/CTC030.codon12_13.vcf.gz
bcftools index /Volumes/T7/GTMBioinfo/vcf/CTC030.codon12_13.vcf.gz
```

I repeat for CTC030-2 and CTC030-6, and optionally for the codon 61 window.

Parse VCFs and compute VAF (template)

I use a small Python snippet to parse the DP/AD/DP4 fields and compute VAF at hotspot positions.

In [ ]:
# Template for parsing bcftools VCFs and computing VAF
import pandas as pd
import gzip

def load_vcf_table(vcf_gz):
    rows = []
    with gzip.open(vcf_gz, 'rt') as f:
        for line in f:
            if line.startswith('#'):
                continue
            chrom, pos, vid, ref, alt, qual, flt, info, fmt, sample = line.strip().split('	', 9)
            d = {'chrom': chrom, 'pos': int(pos), 'ref': ref, 'alt': alt, 'qual': qual, 'filter': flt}
            # Try DP4 (ref_fwd, ref_rev, alt_fwd, alt_rev)
            dp4 = None
            for kv in info.split(';'):
                if kv.startswith('DP4='):
                    try:
                        dp4 = list(map(int, kv.split('=')[1].split(',')))
                    except Exception:
                        dp4 = None
            if dp4 and len(dp4) == 4:
                ref_reads = dp4[0] + dp4[1]
                alt_reads = dp4[2] + dp4[3]
                depth = ref_reads + alt_reads
                vaf = 100.0 * alt_reads / depth if depth else 0.0
            else:
                ref_reads = alt_reads = depth = vaf = None
            d.update({'depth': depth, 'alt_reads': alt_reads, 'vaf_percent': vaf})
            rows.append(d)
    return pd.DataFrame(rows)

# Example usage (adjust paths as needed):
# df = load_vcf_table('/Volumes/T7/GTMBioinfo/vcf/CTC030.codon12_13.vcf.gz')
# print(df.sort_values(['chrom','pos']).to_string(index=False))

Notes and next steps

- This is a minimal, human-readable analysis path. I keep raw command outputs and write plain summaries.
- If a hotspot mutation is detected, I’ll add a simple timeline plot of VAF across timepoints.
- If needed, expand beyond KRAS (e.g., TP53 region on chr17) by swapping the --regions window.